# Predict Success of Youtuber Influences
## Part 3: Prediction models
#### 11/20/2017, Julianne Freeman

In [83]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext version_information
%version_information numpy, scipy, matplotlib, pandas

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 3.6.0 64bit [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
IPython 5.1.0
OS Darwin 16.1.0 x86_64 i386 64bit
numpy 1.11.3
scipy 0.18.1
matplotlib 2.0.0
pandas 0.19.2
Sun Nov 26 22:13:06 2017 EST

In [84]:
from __future__ import division
from datetime import datetime 
import requests
from lxml import html, etree
import json
from textblob import TextBlob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from collections import Counter

#never print matching warnings
warnings.filterwarnings('ignore') 

#set how pandas displays data
pd.options.display.max_columns = 100
pd.options.display.max_rows = 35
pd.options.display.width = 120

#change the max # of rows pandas will display
pd.options.display.max_rows = 4000

#set to display all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#import seaborn to draw pretty graphs
import seaborn as sns
sns.set_style('whitegrid')

# Step 1: Prep variables to used in models

From Part 2, I determined the following variables are likely to have a predictive role in determining viewCount
- type (gaming, family, ect.)
- type_of_day
- title_length
- season

In [141]:
data=pd.read_csv('Youtube_Project_20171121.csv')
data.head(2)

,Unnamed: 0,Unnamed: 0.1,commentCount,dislikeCount,favoriteCount,likeCount,viewCount,video_id,publishedAt,date,week,channelId,title,description,channelTitle,categoryId,key_word,type,duration,hours,minutes,seconds,total_mins,total_secs,year,month,yearmonth,vlog,title_length,description_length,title_channel,month_name,season,day_of_week,type_of_day,series
0,0,0,739.0,191.0,0,11178.0,274412.0,zfmi2JatvlY,2016-06-24 23:22:46,2016-06-24 00:00:00,25,UCDHQbU57NZilrhbuZNbQcRA,WHAT'S IN MY TRAVEL BAG | PatrickStarrr,Hey everyone! I've been traveling a lot lately...,PatrickStarrr,26,NaN,beauty,PT15M,0,15,0,15,900,2016,6,201606,notvlog,39,1516,includes_CT,June,summer,Fri,weekday,no
1,1,1,1010.0,224.0,0,11737.0,319477.0,XlawoU0xCwQ,2016-06-23 02:46:20,2016-06-23 00:00:00,25,UCDHQbU57NZilrhbuZNbQcRA,I AM A GEISHA | JAPAN VLOG | PatrickStarrr,"Discount on TATCHA use code ""PSTARRR"" \nHuge t...",PatrickStarrr,26,NaN,beauty,PT28M32S,0,28,32,28,1712,2016,6,201606,vlog,42,1595,includes_CT,June,summer,Thurs,weekday,no


In [142]:
del data['Unnamed: 0']
del data['Unnamed: 0.1']
data.head(2)

,commentCount,dislikeCount,favoriteCount,likeCount,viewCount,video_id,publishedAt,date,week,channelId,title,description,channelTitle,categoryId,key_word,type,duration,hours,minutes,seconds,total_mins,total_secs,year,month,yearmonth,vlog,title_length,description_length,title_channel,month_name,season,day_of_week,type_of_day,series
0,739.0,191.0,0,11178.0,274412.0,zfmi2JatvlY,2016-06-24 23:22:46,2016-06-24 00:00:00,25,UCDHQbU57NZilrhbuZNbQcRA,WHAT'S IN MY TRAVEL BAG | PatrickStarrr,Hey everyone! I've been traveling a lot lately...,PatrickStarrr,26,NaN,beauty,PT15M,0,15,0,15,900,2016,6,201606,notvlog,39,1516,includes_CT,June,summer,Fri,weekday,no
1,1010.0,224.0,0,11737.0,319477.0,XlawoU0xCwQ,2016-06-23 02:46:20,2016-06-23 00:00:00,25,UCDHQbU57NZilrhbuZNbQcRA,I AM A GEISHA | JAPAN VLOG | PatrickStarrr,"Discount on TATCHA use code ""PSTARRR"" \nHuge t...",PatrickStarrr,26,NaN,beauty,PT28M32S,0,28,32,28,1712,2016,6,201606,vlog,42,1595,includes_CT,June,summer,Thurs,weekday,no


In [87]:
#determine number of rows in data
data.shape[0]

11509

In [108]:
#check for null values
data.isnull().sum()

commentCount             37
dislikeCount             17
favoriteCount             0
likeCount                17
viewCount                12
video_id                  0
publishedAt               0
date                      0
week                      0
channelId                 0
title                     0
description              12
channelTitle              0
categoryId                0
key_word              11193
type                      0
duration                  0
hours                     0
minutes                   0
seconds                   0
total_mins                0
total_secs                0
year                      0
month                     0
yearmonth                 0
vlog                      0
title_length              0
description_length        0
title_channel             0
month_name                0
season                    0
day_of_week               0
type_of_day               0
series                    0
dtype: int64

In [109]:
data['viewCount'].fillna((data['viewCount'].mean()), inplace=True)

In [110]:
# define data sets
#X = data[['type','type_of_day','title_length','season']]
X = data[['type','type_of_day','title_length','season']]
Y = data['viewCount']

# view start of data
X.head(1)
Y.head(1)

# view null values
X.isnull().sum()
Y.isnull().sum()

,type,type_of_day,title_length,season
0,beauty,weekday,39,summer


0    274412.0
Name: viewCount, dtype: float64

type            0
type_of_day     0
title_length    0
season          0
dtype: int64

0

In [111]:
# change categorical into separate columns to not bias the data
X=pd.get_dummies(X, columns=['type'])
X=pd.get_dummies(X, columns=['type_of_day'])
X=pd.get_dummies(X, columns=['season'])

# view start of data
X.head(1)
Y.head(1)

,title_length,type_beauty,type_family,type_fashion,type_food,type_gaming,type_lifestyle,type_news,type_science,type_sports,type_of_day_weekday,type_of_day_weekend,season_Fall,season_spring,season_summer,season_winter
0,39,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


0    274412.0
Name: viewCount, dtype: float64

# Step 2: Run the models

In [112]:
# Break into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

In [136]:
print('Multiple Linear Regression')

# Don't scale the data for linear regression
_='''from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)'''

# linear regression
from sklearn.linear_model import LinearRegression
_=linreg = LinearRegression()
_=linreg.fit(X_train, Y_train)
_=score=linreg.score(X_train, Y_train)
print('Training Set - LR score: %f' % score)

# calculate testing data
from sklearn.metrics import r2_score
y_true, y_pred = Y_test, linreg.predict(X_test)
R2 = r2_score(y_true, y_pred, sample_weight=None, multioutput='uniform_average')
print('Testing Set - R2: %f' % R2)

# get Correlation Coefficient for each feature using Logistic Regression
_='''coeff_df = pd.DataFrame(X_train.columns.delete(0))
coeff_df.columns = ['Features']
coeff_df["Coefficient Estimate"] = pd.Series(logreg.coef_[0])
coeff_df'''

Multiple Linear Regression
Training Set - LR score: 0.240617
Testing Set - R2: 0.171281


In [137]:
print('Support Vector Regression')

# Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train_scale = scaler.transform(X_train)
X_test_scale = scaler.transform(X_test) #need to scale test data the same way that train data was scaled

# Support Vector Regression
from sklearn.svm import SVR
_=regressor=SVR(kernel='rbf') #regressor is an object of the module
_=regressor.fit(X_train_scale, Y_train) #use train data to determine regressor
_=score=regressor.score(X_train_scale, Y_train)
print('Training Set - SVR score: %f' % score)

# calculate testing data
from sklearn.metrics import r2_score
_=y_true, y_pred = Y_test, regressor.predict(X_test_scale)
R2 = r2_score(y_true, y_pred, sample_weight=None, multioutput='uniform_average')
print('Testing Set - R2: %f' % R2)

Support Vector Regression
Training Set - SVR score: -0.164602
Testing Set - R2: -0.117321


In [138]:
print('Decision Tree Regressor')

# Don't scale the data with Random Forest
_='''from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)'''

# Random Forest
from sklearn.tree import DecisionTreeRegressor
_=decision_tree = DecisionTreeRegressor(criterion='mse') #mse = mean square error
_=decision_tree.fit(X_train, Y_train)
_=score=decision_tree.score(X_train, Y_train)
print('Training Set - DRT score: %f' % score)

# calculate testing data
from sklearn.metrics import r2_score
_=y_true, y_pred = Y_test, decision_tree.predict(X_test_scale)
R2 = r2_score(y_true, y_pred, sample_weight=None, multioutput='uniform_average')
print('Testing Set - R2: %f' % R2)

Decision Tree Regressor
Training Set - DRT score: 0.580265
Testing Set - R2: -0.023769


In [139]:
print('K Neighbors Regressor')

# Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train_scale = scaler.transform(X_train)
X_test_scale = scaler.transform(X_test)

#K Neighbors Classifier
from sklearn.neighbors import KNeighborsRegressor
_=knr = KNeighborsRegressor(n_neighbors = 3)
_=knr.fit(X_train_scale, Y_train)
_=score=knr.score(X_train_scale, Y_train)
print('Training Set - K Neighbors score: %f' % score)

# calculate testing data
from sklearn.metrics import r2_score
_=y_true, y_pred = Y_test, knr.predict(X_test_scale)
R2 = r2_score(y_true, y_pred, sample_weight=None, multioutput='uniform_average')
print('Testing Set - R2: %f' % R2)

K Neighbors Regressor
Training Set - K Neighbors score: 0.407170
Testing Set - R2: 0.132845


# Step 4: Results

Thoughts: R2 values are all low for test data sets

See Part 4